In [11]:
import pandas as pd
from datetime import datetime
import rqalpha as rqa
import rqalpha_plus
import rqdatac as rqd
import rqoptimizer as rqo

rqd.init()

F:\anaconda\envs\ricequant\lib\site-packages\rqdatac\client.py:190: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


In [8]:
def get_300_enhanced_universe(date,stock_num):
    score = get_alpha_score(date)
     # 中证800成分股
    component = rqd.index_components('000906.XSHG', date)
    score_index = score.index.tolist()
    # 取交集
    score = score.loc[list(set(component).intersection(score_index))]
    target_stock = score.sort_values(ascending=False)[:stock_num].index.tolist()
    return target_stock, score

In [9]:
def get_300_enhanced_portfolio(target_stock,date,score,benchmark='000300.XSHG'):
    
    # define objective and constrains according to specification
    objective = ...
    
    # industry constraints
    industry_constraints = ...
    
    # style constraints
    style_constraints = ...
    
    # turnover constraints
    current_holding = context.positions 
    turnover_contraints = rqo.TurnoverLimit(current_holding, ...)
    
    constraints = [industry_constraints, style_constraints, turnover_contraints]
    
    portfolio = rqo.portfolio_optimize(date, benchmark, objective, constraints, ...) 
    return portfolio.loc[lambda x:x!=0]


# for backtest
def handle_bar_300_enhanced(context, bar_dict):
    if not _should_rebalance(context):
        return
    today = context.now.strftime('%Y%m%d')
    # 目标股和分数
    target_stock,score = get_300_enhanced_universe(today,context.stock_num)
    # 目标权重
    target_portfolio = get_300_enhanced_portfolio(target_stock,today,score)
    # 调仓函数
    rebalance(context,bar_dict,target_portfolio)

In [ ]:
# 假设股票得分在当日开盘前能获取
def before_trading(context):
    score = get_alpha_score(context.now.date(), context.alpha_score_path)
    
    # 若当日的读取的alpha分数非空, 则执行调仓
    should_rebalance = (not score.empty)
    if should_rebalance:
        # 对于股票回测, 执行before_trading时, 当前时间是早上8:00
        # 在此刻通过context获取的持仓权重, 是基于上一天收盘价计算得到的
        context.target_portfolio = get_300_enhanced_portfolio(context)
    else:
        context.target_portfolio = None
        
        
def handle_bar(context, bar_dict):
    # 对于股票回测, 执行handle_bar, 当前时间是15:00
    # 在此刻通过context获取的持仓权重, 是基于当前这根bar的收盘价计算得到的
    if context.target_portfolio is not None:
        rebalance()